# Few-shot Learning Comparison

## Imports

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.svm import SVC

import torch
from torchtext.datasets import AG_NEWS, IMDB
from data.autoencoding_dataset import AutoencodingDataset

## Preparing Dataset

In [2]:
def prepare_dataset(dataset=AG_NEWS(split='test')):
    dr = list(dataset)
    sentences = [d[1] for d in dr]
    classes = [d[0] for d in dr]
    dp = AutoencodingDataset(sentences, classes=classes, max_len=16)
    dp.preproces()
    return dp

/home/bodo/.local/lib/python3.8/site-packages/torch/utils/data/datapipes/utils/common.py:24: UserWarning: Lambda function is not supported for pickle, please use regular python function or functools.partial instead.
  warnings.warn(


## Loading Encoder

In [3]:
def load_encoder(path):
    m = Autoencoder(100,100,100,4,3,variational=True, max_log2len=8)
    m.load_state_dict(torch.load(path_to_model, map_location=torch.device('cpu')))
    encoder = m.encoder
    return encoder

# SVM Comparison Script

In [4]:
def prepare_comparison(encoder, dp):
  
  dl = torch.utils.data.DataLoader(dp, batch_size=1, shuffle=True)
  
  x_word_embeded = []
  x_emb = []
  x_bow = []
  y = []
  for x, _ in zip(dl, range(1100)): #ten range wywalic
    x_bow.append(x[0].data.mean(1).numpy().reshape(-1))
    x_word_embeded.append(x[0].data.numpy().reshape(-1))
    x_emb.append(encoder(x[0]).data.numpy().reshape(-1))
    y.append(int(x[1].data[0]))

  acc_embeded = []
  acc_word_embeded = []
  acc_bow = []
  for i in tqdm(range(10,1000, 10)):
    clf_embeded = SVC()
    clf_word_embeded = SVC()
    clf_bow = SVC()
    
    clf_embeded.fit(x_emb[:i], y[:i])
    clf_word_embeded.fit(x_word_embeded[:i], y[:i])
    clf_bow.fit(x_bow[:i], y[:i])
    acc_embeded.append(accuracy_score(y[i:], clf_embeded.predict(x_emb[i:])))
    acc_word_embeded.append(accuracy_score(y[i:], clf_word_embeded.predict(x_word_embeded[i:])))
    acc_bow.append(accuracy_score(y[i:], clf_bow.predict(x_bow[i:])))

  return acc_embeded, acc_word_embeded, acc_bow

# Comparison

In [ ]:
encoder = load_encoder('')

In [ ]:
dp = prepare_dataset()

In [ ]:
acc_embeded, acc_word_embeded, acc_bow = prepare_comparison(encoder, dp)